In [247]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

pd.options.display.max_columns = 200

1. Data Preprocessing (Check Null , Data Type , See The correct Form ofthe data if there any white space for example or not)
2. Check And Handel The Outliers Using (EX : IQR OR Z SCORE ) 
3. Feature Engineering ( Feature Selection , Feature Extraction)
4. Transformation For The Categorical Data 
5. Train Test Split
6. Modeling And Accuracy Calculation

In [248]:
df = pd.read_csv('first inten project.csv')
df.head()

,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,type of meal,car parking space,room type,lead time,market segment type,repeated,P-C,P-not-C,average price,special requests,date of reservation,booking status
0,INN00001,1,1,2,5,Meal Plan 1,0,Room_Type 1,224,Offline,0,0,0,88.00,0,10/2/2015,Not_Canceled
1,INN00002,1,0,1,3,Not Selected,0,Room_Type 1,5,Online,0,0,0,106.68,1,11/6/2018,Not_Canceled
2,INN00003,2,1,1,3,Meal Plan 1,0,Room_Type 1,1,Online,0,0,0,50.00,0,2/28/2018,Canceled
3,INN00004,1,0,0,2,Meal Plan 1,0,Room_Type 1,211,Online,0,0,0,100.00,1,5/20/2017,Canceled
4,INN00005,1,0,1,2,Not Selected,0,Room_Type 1,48,Online,0,0,0,77.00,0,4/11/2018,Canceled


In [249]:
df = df.rename(columns={
    'P-C': 'reserved then cancelled',
    'P-not-C': 'reserved then not cancelled',
    'lead time': 'days from booking to arrival',
    'repeated': 'visited before',
    'average price ': 'average price',
    'market segment type': 'market type'})

df.head()

,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,type of meal,car parking space,room type,days from booking to arrival,market type,visited before,reserved then cancelled,reserved then not cancelled,average price,special requests,date of reservation,booking status
0,INN00001,1,1,2,5,Meal Plan 1,0,Room_Type 1,224,Offline,0,0,0,88.00,0,10/2/2015,Not_Canceled
1,INN00002,1,0,1,3,Not Selected,0,Room_Type 1,5,Online,0,0,0,106.68,1,11/6/2018,Not_Canceled
2,INN00003,2,1,1,3,Meal Plan 1,0,Room_Type 1,1,Online,0,0,0,50.00,0,2/28/2018,Canceled
3,INN00004,1,0,0,2,Meal Plan 1,0,Room_Type 1,211,Online,0,0,0,100.00,1,5/20/2017,Canceled
4,INN00005,1,0,1,2,Not Selected,0,Room_Type 1,48,Online,0,0,0,77.00,0,4/11/2018,Canceled


In [250]:
df.duplicated().sum()

0

In [251]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
skewed = df[numeric_cols].skew().sort_values(ascending=False)
skewed_features = skewed[abs(skewed) > 0.75].index
print("Features with high skewness:", list(skewed_features))

for col in skewed_features:
    if (df[col] > 0).all():
        df[col] = np.log1p(df[col])

Features with high skewness: ['reserved then cancelled', 'reserved then not cancelled', 'visited before', 'car parking space', 'number of children', 'number of week nights', 'days from booking to arrival', 'special requests']


numerical data: number of adults, number of children, number of weekend nights	, number of week nights , avg price , days from booking to arrival , reserved then not cancelled , reserved then cancelled , special requests

In [252]:
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower) | (data[column] > upper)]
    return outliers

print("Available columns:", df.columns.tolist())

columns_to_check = [
    'number of adults',
    'number of children',
    'number of weekend nights',
    'number of week nights',
    'days from booking to arrival',
    'reserved then cancelled',
    'reserved then not cancelled',
    'special requests',
    'average price'
]

for i in columns_to_check:
        outliers = detect_outliers_iqr(df, i)
        print(f'Outliers in "{i}": {len(outliers)} rows')


Available columns: ['Booking_ID', 'number of adults', 'number of children', 'number of weekend nights', 'number of week nights', 'type of meal', 'car parking space', 'room type', 'days from booking to arrival', 'market type', 'visited before', 'reserved then cancelled', 'reserved then not cancelled', 'average price', 'special requests', 'date of reservation', 'booking status']
Outliers in "number of adults": 10175 rows
Outliers in "number of children": 2702 rows
Outliers in "number of weekend nights": 21 rows
Outliers in "number of week nights": 324 rows
Outliers in "days from booking to arrival": 1332 rows
Outliers in "reserved then cancelled": 338 rows
Outliers in "reserved then not cancelled": 812 rows
Outliers in "special requests": 762 rows
Outliers in "average price": 1696 rows


In [253]:
df.describe()

,number of adults,number of children,number of weekend nights,number of week nights,car parking space,days from booking to arrival,visited before,reserved then cancelled,reserved then not cancelled,average price,special requests
count,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000,36285.000000
mean,1.844839,0.105360,0.810693,2.204602,0.030977,85.239851,0.025630,0.023343,0.153369,103.421636,0.619733
std,0.518813,0.402704,0.870590,1.410946,0.173258,85.938796,0.158032,0.368281,1.753931,35.086469,0.786262
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,17.000000,0.000000,0.000000,0.000000,80.300000,0.000000
50%,2.000000,0.000000,1.000000,2.000000,0.000000,57.000000,0.000000,0.000000,0.000000,99.450000,0.000000
75%,2.000000,0.000000,2.000000,3.000000,0.000000,126.000000,0.000000,0.000000,0.000000,120.000000,1.000000
max,4.000000,10.000000,7.000000,17.000000,1.000000,443.000000,1.000000,13.000000,58.000000,540.000000,5.000000


In [254]:
#clipping outliers using IQR method
def cap_outliers_iqr(data, columns):
    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        num_below = (data[col] < lower).sum()
        num_above = (data[col] > upper).sum()

        data[col] = data[col].clip(lower, upper)

        print(f'[{col}] - Capped {num_below} below and {num_above} above to IQR limits.')

    return data


df = cap_outliers_iqr(df, columns_to_check)


[number of adults] - Capped 7841 below and 2334 above to IQR limits.
[number of children] - Capped 0 below and 2702 above to IQR limits.
[number of weekend nights] - Capped 0 below and 21 above to IQR limits.
[number of week nights] - Capped 0 below and 324 above to IQR limits.
[days from booking to arrival] - Capped 0 below and 1332 above to IQR limits.
[reserved then cancelled] - Capped 0 below and 338 above to IQR limits.
[reserved then not cancelled] - Capped 0 below and 812 above to IQR limits.
[special requests] - Capped 0 below and 762 above to IQR limits.
[average price] - Capped 627 below and 1069 above to IQR limits.


In [255]:
df.dtypes

Booking_ID                       object
number of adults                  int64
number of children                int64
number of weekend nights          int64
number of week nights             int64
type of meal                     object
car parking space                 int64
room type                        object
days from booking to arrival    float64
market type                      object
visited before                    int64
reserved then cancelled           int64
reserved then not cancelled       int64
average price                   float64
special requests                float64
date of reservation              object
booking status                   object
dtype: object

In [256]:
#Map values
df['booking status'] = df['booking status'].map({
    'Not_Canceled': 0,
    'Canceled': 1
})

In [257]:

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
#rmeove booking state from them

encoder = OneHotEncoder(handle_unknown='ignore')


low_cardinality_cols = [col for col in categorical_cols if df[col].nunique() < 15]


encoder.fit(df[low_cardinality_cols])

# Step 2: Transform
encoded_data = encoder.transform(df[low_cardinality_cols])

# Step 3: Get column names
encoded_col_names = encoder.get_feature_names_out(low_cardinality_cols)

# Step 4: Drop and insert in-place
df.drop(columns=low_cardinality_cols, inplace=True)

# Assuming `encoded_data` is a sparse matrix from OneHotEncoder
encoded_dense = encoded_data.toarray()

# Create column-wise DataFrame from dense encoded data
for i, col_name in enumerate(encoded_col_names):
    df[col_name] = encoded_dense[:, i]  # This will match the index correctly


In [258]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36285 entries, 0 to 36284
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Booking_ID                    36285 non-null  object 
 1   number of adults              36285 non-null  int64  
 2   number of children            36285 non-null  int64  
 3   number of weekend nights      36285 non-null  int64  
 4   number of week nights         36285 non-null  int64  
 5   car parking space             36285 non-null  int64  
 6   days from booking to arrival  36285 non-null  float64
 7   visited before                36285 non-null  int64  
 8   reserved then cancelled       36285 non-null  int64  
 9   reserved then not cancelled   36285 non-null  int64  
 10  average price                 36285 non-null  float64
 11  special requests              36285 non-null  float64
 12  date of reservation           36285 non-null  object 
 13  b

In [259]:
df.columns

Index(['Booking_ID', 'number of adults', 'number of children',
       'number of weekend nights', 'number of week nights',
       'car parking space', 'days from booking to arrival', 'visited before',
       'reserved then cancelled', 'reserved then not cancelled',
       'average price', 'special requests', 'date of reservation',
       'booking status', 'type of meal_Meal Plan 1',
       'type of meal_Meal Plan 2', 'type of meal_Meal Plan 3',
       'type of meal_Not Selected', 'room type_Room_Type 1',
       'room type_Room_Type 2', 'room type_Room_Type 3',
       'room type_Room_Type 4', 'room type_Room_Type 5',
       'room type_Room_Type 6', 'room type_Room_Type 7',
       'market type_Aviation', 'market type_Complementary',
       'market type_Corporate', 'market type_Offline', 'market type_Online'],
      dtype='object')

transfer categorical data into numerical

In [260]:
df['date of reservation'] = pd.to_datetime(df['date of reservation'], errors='coerce')
df.head()

,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,car parking space,days from booking to arrival,visited before,reserved then cancelled,reserved then not cancelled,average price,special requests,date of reservation,booking status,type of meal_Meal Plan 1,type of meal_Meal Plan 2,type of meal_Meal Plan 3,type of meal_Not Selected,room type_Room_Type 1,room type_Room_Type 2,room type_Room_Type 3,room type_Room_Type 4,room type_Room_Type 5,room type_Room_Type 6,room type_Room_Type 7,market type_Aviation,market type_Complementary,market type_Corporate,market type_Offline,market type_Online
0,INN00001,2,0,2,5,0,224.0,0,0,0,88.00,0.0,2015-10-02,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,INN00002,2,0,1,3,0,5.0,0,0,0,106.68,1.0,2018-11-06,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,INN00003,2,0,1,3,0,1.0,0,0,0,50.00,0.0,2018-02-28,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,INN00004,2,0,0,2,0,211.0,0,0,0,100.00,1.0,2017-05-20,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,INN00005,2,0,1,2,0,48.0,0,0,0,77.00,0.0,2018-04-11,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [261]:
invalid_dates = df[df['date of reservation'].isna()]
print(f"Number of corrupted or invalid dates: {len(invalid_dates)}")

df = df.dropna(subset=['date of reservation'])

Number of corrupted or invalid dates: 37


In [262]:
df['reservation_day'] = df['date of reservation'].dt.day
df['reservation_month'] = df['date of reservation'].dt.month
df['reservation_weekday'] = df['date of reservation'].dt.weekday 

df.drop(columns=['date of reservation'], inplace=True)
df.head()


,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,car parking space,days from booking to arrival,visited before,reserved then cancelled,reserved then not cancelled,average price,special requests,booking status,type of meal_Meal Plan 1,type of meal_Meal Plan 2,type of meal_Meal Plan 3,type of meal_Not Selected,room type_Room_Type 1,room type_Room_Type 2,room type_Room_Type 3,room type_Room_Type 4,room type_Room_Type 5,room type_Room_Type 6,room type_Room_Type 7,market type_Aviation,market type_Complementary,market type_Corporate,market type_Offline,market type_Online,reservation_day,reservation_month,reservation_weekday
0,INN00001,2,0,2,5,0,224.0,0,0,0,88.00,0.0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2,10,4
1,INN00002,2,0,1,3,0,5.0,0,0,0,106.68,1.0,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6,11,1
2,INN00003,2,0,1,3,0,1.0,0,0,0,50.00,0.0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,28,2,2
3,INN00004,2,0,0,2,0,211.0,0,0,0,100.00,1.0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20,5,5
4,INN00005,2,0,1,2,0,48.0,0,0,0,77.00,0.0,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11,4,2


In [263]:
df.drop(columns=['Booking_ID'], inplace=True)

In [264]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36248 entries, 0 to 36284
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   number of adults              36248 non-null  int64  
 1   number of children            36248 non-null  int64  
 2   number of weekend nights      36248 non-null  int64  
 3   number of week nights         36248 non-null  int64  
 4   car parking space             36248 non-null  int64  
 5   days from booking to arrival  36248 non-null  float64
 6   visited before                36248 non-null  int64  
 7   reserved then cancelled       36248 non-null  int64  
 8   reserved then not cancelled   36248 non-null  int64  
 9   average price                 36248 non-null  float64
 10  special requests              36248 non-null  float64
 11  booking status                36248 non-null  int64  
 12  type of meal_Meal Plan 1      36248 non-null  float64
 13  type o

In [265]:
df['weekends to total'] = df['number of weekend nights'] * 1.0 / (df['number of weekend nights'] + df['number of week nights'])
df['weekdays to total'] = df['number of week nights'] * 1.0 / (df['number of weekend nights'] + df['number of week nights'])
df['weekends to total'] = df['weekends to total'].fillna(0) 
df['weekdays to total'] = df['weekdays to total'].fillna(0)


df.drop(columns=['number of weekend nights', 'number of week nights'], inplace=True)

df['weekends to total'].value_counts()

weekends to total
0.000000    16876
0.500000     4226
0.333333     4213
0.666667     2466
0.250000     2310
1.000000     2305
0.400000     1700
0.285714      974
0.200000      743
0.166667      310
0.454545       55
0.375000       53
0.444444       17
Name: count, dtype: int64

check multicolleniarity

In [266]:
df.drop(columns=[
    'number of adults',
    'number of children',
    'reserved then cancelled',
    'reserved then not cancelled',
    'weekdays to total' 
], inplace=True)

In [267]:
df.columns

Index(['car parking space', 'days from booking to arrival', 'visited before',
       'average price', 'special requests', 'booking status',
       'type of meal_Meal Plan 1', 'type of meal_Meal Plan 2',
       'type of meal_Meal Plan 3', 'type of meal_Not Selected',
       'room type_Room_Type 1', 'room type_Room_Type 2',
       'room type_Room_Type 3', 'room type_Room_Type 4',
       'room type_Room_Type 5', 'room type_Room_Type 6',
       'room type_Room_Type 7', 'market type_Aviation',
       'market type_Complementary', 'market type_Corporate',
       'market type_Offline', 'market type_Online', 'reservation_day',
       'reservation_month', 'reservation_weekday', 'weekends to total'],
      dtype='object')

In [268]:
X = df.drop(columns=['booking status'])
y = df['booking status']

In [269]:
from sklearn.model_selection import train_test_split

X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [270]:
#knn model

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report


knn_model = KNeighborsClassifier(n_neighbors=5 , weights='distance', algorithm='auto')
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.82
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      4884
           1       0.76      0.68      0.72      2366

    accuracy                           0.82      7250
   macro avg       0.80      0.79      0.79      7250
weighted avg       0.82      0.82      0.82      7250



In [271]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=5000)
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy: {accuracy:.2f}")
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred))

Logistic Regression Accuracy: 0.80
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.88      0.86      4884
           1       0.72      0.63      0.67      2366

    accuracy                           0.80      7250
   macro avg       0.78      0.76      0.76      7250
weighted avg       0.80      0.80      0.80      7250



In [272]:
from sklearn.tree import DecisionTreeClassifier

decision_tree_model = DecisionTreeClassifier(random_state=0)
decision_tree_model.fit(X_train, y_train)
y_pred = decision_tree_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree Accuracy: {accuracy:.2f}")
print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred))

Decision Tree Accuracy: 0.87
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.90      4884
           1       0.80      0.82      0.81      2366

    accuracy                           0.87      7250
   macro avg       0.85      0.86      0.86      7250
weighted avg       0.87      0.87      0.87      7250



In [273]:
#random forest
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(random_state=0)
random_forest_model.fit(X_train, y_train)
y_pred = random_forest_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy:.2f}")
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred))

Random Forest Accuracy: 0.90
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      4884
           1       0.88      0.81      0.84      2366

    accuracy                           0.90      7250
   macro avg       0.90      0.88      0.89      7250
weighted avg       0.90      0.90      0.90      7250



In [274]:
#after encoding 0 cancel and 1 keep
print("Encoded DataFrame:")
print(df.head())


Encoded DataFrame:
   car parking space  days from booking to arrival  visited before  \
0                  0                         224.0               0   
1                  0                           5.0               0   
2                  0                           1.0               0   
3                  0                         211.0               0   
4                  0                          48.0               0   

   average price  special requests  booking status  type of meal_Meal Plan 1  \
0          88.00               0.0               0                       1.0   
1         106.68               1.0               0                       0.0   
2          50.00               0.0               1                       1.0   
3         100.00               1.0               1                       1.0   
4          77.00               0.0               1                       0.0   

   type of meal_Meal Plan 2  type of meal_Meal Plan 3  \
0                     

In [ ]:
#save encoder , and models
import joblib
joblib.dump(encoder, 'encoder.pkl')
joblib.dump(random_forest_model, 'rf_model.pkl')


['rf_model.pkl']